# fn1 get_matchtable

In [68]:
from tqdm.auto import tqdm
import polars as pl
import glob
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import xlwings as xw
import time
import random

csv_target = r".\csv\3_all_matches.csv"
csv_file=Path(csv_target)

def fn_get_matchtable(url,index):
        counter_matchinseason=0
        #exists = 'final_df' in globals() and isinstance(globals()['final_df'], pl.DataFrame)
        #print("final_df:", exists)
        global counter
        #print(counter)
        totalheader = ["Country", "Comp", "Season",  "Round", "Wk", "Day", "Date", "Time", "Home", "xG_h", "Score", "xG_a", "Away", 
                       "Attendance", "Venue", "Referee","href","CoachH", "CoachA", "CaptainH", "CaptainA", "url_CaptainH", "url_CaptainA",
                       "goal1","goal2","goal3","goal4","goal5","goal6","goal7","goal8","goal9","goal10","goal11","goal12","goal13","goal14",
                       "goal15","goal16","goal17","goal18","goal19","goal20","goal21","goal22","goal23","goal24","goal25"]
        # 1 get soup
        user_agent1 = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        user_agent2="Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
        headers = {'User-Agent': user_agent1}
        counter+=1
        # print("fn1",counter)
        time.sleep(random.uniform(1, 3))
        response = requests.get(url,headers=headers)
        webpage = response.content
        #print(webpage)
        soup = BeautifulSoup(webpage, 'html.parser')
 
        # 2 Get Generals
        # 2a Liganame
        span_title = soup.find('h2')
        if span_title:
                span = span_title.find('span')
                if span:
                        span_text = span.text
                        first_space_index = span_text.find(' ')
                        Liga = span_text[first_space_index + 1:] 
                else:
                        print(f"No span in h2 for {url}")
                        return None
        else:
                print(f"No Tabletitle for {url}")
                return None
        #print("Liga:", Liga)
        
        # 2b Countryname
        if not soup.find('strong', string="Governing Country"):
                Country = ""
        else:
                Country = soup.find('strong', string="Governing Country").find_next('a').text
        #print("country",Country)        

        # 2c Seasonname
        prevnext_exists = soup.find("div", class_="prevnext")
        if prevnext_exists:
                h1_sibling = prevnext_exists.find_previous_sibling("h1")
                if h1_sibling:
                        Season = h1_sibling.text.split(" ")[0].strip()
                else:
                        print("No h1 sibling found before the 'prevnext' class.")
        else:
                print("No 'prevnext' class found.")
        # print("Season:", Season)


        # 3 GET TABLE & HEADER 
        table = soup.find('table')
        if not table:
                print(f"No table for {url}")
                return None
        # flags
        flags = table.find_all('span', class_=lambda value: value and value.startswith('f-i'))

        # headers
        header = table.find('thead').tr 
        headers  = [th.get_text(strip=True) for th in header.find_all('th')]
        #print(headers)
        
        # duplicate im header korrigieren
        xg_count = 0
        for i, header in enumerate(headers):
                if header == 'xG':
                        xg_count += 1
                        if xg_count == 1:
                                headers[i] = 'xG_h'
                        elif xg_count == 2: 
                                headers[i] = 'xG_a'
        
        headers +=["href","CoachH", "CoachA", "CaptainH", "CaptainA", "url_CaptainH", "url_CaptainA",
                   "goal1","goal2","goal3","goal4","goal5","goal6","goal7","goal8","goal9","goal10",
                   "goal11","goal12","goal13","goal14","goal15","goal16","goal17","goal18","goal19","goal20",
                   "goal21","goal22","goal23","goal24","goal25"]
        #print("headers:",headers)

        score_index = headers.index('Score') 
        # print("score_index", score_index)

        rows_list = []
        base_url = "https://fbref.com"
        row_counter= 1
    


# Assuming fn_get_matchdetails(href) is properly defined to fetch and return detailed information.
        
        for row in table.find_all('tr')[1:]:  # Skip the header row
                cells = row.find_all(['th', 'td'])
                row_data = [cell.text.strip() for cell in cells]
        
                # print("row_data",row_data)
                # print("cells:", len(cells),"score_index", score_index)
                if len(cells) > score_index:  # Check if the cell exists
                        td_score = cells[score_index]
                        a_tag = td_score.find('a')
                        
                        if a_tag and a_tag.has_attr('href'):
                                href = base_url + a_tag['href']
                                # random pause
                                time.sleep(random.uniform(1, 3) )  
                                # save data
                                counter+=1
                                counter_matchinseason+=1
                                # print("fn2", counter)
                                mdetails = fn_get_matchdetails(href)
                                mdetails =list(mdetails.values())
                                # print("Mdetails", mdetails)
                                # print(counter, href)
                                # print(counter,mdetails)
                                row_data.append(href)  # Add the href to row data
                                row_data.extend(mdetails)  # Extend row_data with details from mdetails
                                # row_fillers += [""] * (len(headers) - len(row_data))
                                # row_data.append(row_fillers)
                                row_fillers = [""] * (len(headers) - len(row_data))
                                row_data.extend(row_fillers) 
                                #print(counter, "len", len(row_data), len(headers))
                        else:
                                href = None  # Explicitly setting href to None if not found
                        
                else:
                        print(f"Row does not have a cell at index {score_index}. Skipping href and details extraction.")
                
                #print("row",index, len(row_data),len(headers))
                if len(row_data) == len(headers):
                        print((f"status: {response.status_code}, id {index +1}.{counter_matchinseason}_{counter}"), "season:", url)
                        rows_list.append(row_data)  
        # DATAFRAME
        df = pl.DataFrame(rows_list, schema=headers) 
        #print(df)

        # # flaggen entfernen
        if flags:
                df = df.with_columns(pl.col('Away').str.slice(3, None).alias('Away'))
                df = df.with_columns(pl.col('Home').str.replace(r'.{3}$', '').alias('Home'))

        # Add missing columns from total_header as None (but will eventually be strings)
        for col in totalheader:
                if col not in df.columns:
                        df = df.with_columns(pl.lit(None).cast(pl.Utf8).alias(col))

        # sequence of columns like in TotalHeader
        #df = df.select([pl.col(col).cast(pl.Utf8) for col in totalheader])
        df = df.select(totalheader)


        # fill additional standard infos like country and competition ...
        df = df.with_columns(pl.lit(Country).alias("Country"))
        df = df.with_columns(pl.lit(Liga).alias("Comp"))  
        df = df.with_columns(pl.lit(Season).alias("Season"))      
        return df


# EXAMPLE ###############################################################################################

# TOTALHEADER
totalheader = ["Country", "Comp", "Season",  "Round", "Wk", "Day", "Date", "Time", "Home", "xG_h", "Score", "xG_a", "Away", "Attendance", "Venue", "Referee","href","CoachH", "CoachA", "CaptainH", "CaptainA", "url_CaptainH", "url_CaptainA","goal1","goal2","goal3","goal4","goal5","goal6","goal7","goal8","goal9","goal10","goal11","goal12","goal13","goal14","goal15","goal16","goal17","goal18","goal19","goal20",
"goal21","goal22","goal23","goal24","goal25"]
df_totalheader = pl.DataFrame({i: [] for i in totalheader})
df_totalheader

counter =-1
index = 0
#url = 'https://fbref.com/en/comps/14/2016/schedule/'
#url ="https://fbref.com/en/comps/43/schedule/Veikkausliiga-Scores-and-Fixtures"
url="https://fbref.com/en/comps/189/2018-2019/schedule/2018-2019-Womens-Super-League-Scores-and-Fixtures"
url ="https://fbref.com/en/comps/14/2024/schedule/"

df= fn_get_matchtable(url,index)
df

#xw.view(df)

# # KONVERTIERUNGEN ##############################################################
# # if df_total.dtypes[df_total.columns.index("Date")] != pl.Date:
# #     df_total = df_total.with_columns( df_total["Date"].str.strptime(pl.Date, "%d/%m/%Y", strict=False).alias("Date") )

# # if "Time" in df_total.columns and df_total.dtypes[df_total.columns.index("Time")] != pl.Datetime:
# #     df_total = df_total.with_columns(df_total["Time"].str.strip_chars().str.strptime(pl.Time, "%H:%M").alias("Time") )


# # falls ich die punkte gegen kommas tauschen möchte
# # for col_name in df_total.columns:
# #     if df_total[col_name].dtype == pl.Utf8:
# #         df_total = df_total.with_column( df_total[col_name].str.replace(".", ",", global_=True).alias(col_name))

status: 200, id 1.1_1  season: https://fbref.com/en/comps/14/2024/schedule/


KeyboardInterrupt: 

# fn2 get_details

In [25]:
# fn_match_details
import requests
from bs4 import BeautifulSoup

def fn_get_matchdetails(url):
    user_agent1 = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    user_agent2="Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
    headers = {'User-Agent': user_agent2}
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    datapoints = soup.find_all('div', class_='datapoint')
    
    captains=[]
    coaches=[]
    captains_links=[]
    playerH_links=[]
    playerA_links=[]
    
    captains = [dp.get_text(strip=True) for dp in datapoints if 'Captain' in dp.get_text()]

    for dp in datapoints:
        if 'Captain' in dp.get_text():
            captain_a_tag = dp.find('a')  # Find the <a> tag within the datapoint
            if captain_a_tag:
                captains_links.append("fbref.com" + captain_a_tag['href']) 
    
    coaches = [dp.get_text(strip=True) for dp in datapoints if 'Manager' in dp.get_text()]

    goalsH = []
    team_events = soup.find('div', id="a")
    if team_events:
        for event in team_events.find_all('div', recursive=False):
            if event.find('div', class_='event_icon penalty_goal') or event.find('div', class_='event_icon goal'):
                player_name = event.find('a').text if event.find('a') else 'Unknown Player'
                player_link_tag = event.find('a')
                playerH_link = "fbref.com" + player_link_tag['href'] if player_link_tag else 'No link'
                event_text = event.get_text()
                time = event_text.split('·')[-1].strip()
                time = time[:-1]  # Removes the last character, typically an apostrophe
                goalsH.append(f"H_{time}_{player_name}_{playerH_link}")
                playerH_links.append({playerH_link})
        playerH_links = [next(iter(l)) for l in playerH_links]

    
    goalsA = []
    team_events = soup.find('div', id="b")
    if team_events:
        for event in team_events.find_all('div', recursive=False):
            if event.find('div', class_='event_icon penalty_goal') or event.find('div', class_='event_icon goal'):
                player_name = event.find('a').text if event.find('a') else 'Unknown Player'
                player_link_tag = event.find('a')
                playerA_link = "fbref.com" + player_link_tag['href'] if player_link_tag else 'No link'
                event_text = event.get_text()
                time = event_text.split('·')[-1].strip()
                time = time[:-1]  # Removes the last character, typically an apostrophe
                goalsA.append(f"A_{time}_{player_name}_{playerA_link} ")
                playerH_links.append({playerA_link})
        playerA_links = [next(iter(l)) for l in playerA_links]

    goals=[goalsH + goalsA]
    goals = goals[0]
    mdetails_dict = {}
    #print(len(captains))
    mdetails_dict = {
        "CoachH": coaches[0].replace("Manager: ", "") if len(coaches) > 0 else "",
        "CoachA": coaches[1].replace("Manager: ", "") if len(coaches) > 0 else "",
        "CaptainH": captains[0].replace("Captain:", "") if len(captains) > 0 else "",
        "CaptainA": captains[1].replace("Captain:", "") if len(captains) > 0 else "",
        "url_CaptainH": captains_links[0] if len(captains)> 0 else "",
        "url_CaptainA": captains_links[1] if len(captains)> 0 else "",
    }
    
    for i, goal in enumerate(goals):
        mdetails_dict[f"goal{i+1}"] = goal if i < len(goals) else ""

    return mdetails_dict

# EXAMPLE -------------------------------------------------------------------------------------------------------
url = "https://fbref.com/en/matches/0aa27858/Always-Ready-Sporting-Cristal-February-20-2024-Copa-Libertadores"
mdetails_dict = fn_get_matchdetails(url)
mdetails_dict

{'CoachH': 'Óscar\xa0Villegas',
 'CoachA': 'Enderson\xa0Moreira',
 'CaptainH': 'Luis\xa0Caicedo',
 'CaptainA': 'Yoshimar\xa0Yotún',
 'url_CaptainH': 'fbref.com/en/players/30521c1c/Luis-Caicedo',
 'url_CaptainA': 'fbref.com/en/players/b6af8c85/Yoshimar-Yotun',
 'goal1': 'H_14_Wesley Tanque_fbref.com/en/players/0830755e/Wesley-Tanque',
 'goal2': 'H_48_Dorny Romero_fbref.com/en/players/b2694e42/Dorny-Romero',
 'goal3': 'H_54_Robson Tome de Araujo_fbref.com/en/players/6c5b59c5/Robson-Tome-de-Araujo',
 'goal4': 'H_59_Héctor Cuellar_fbref.com/en/players/44aa5f6e/Hector-Cuellar',
 'goal5': 'H_78_Dorny Romero_fbref.com/en/players/b2694e42/Dorny-Romero',
 'goal6': 'H_89_Moises Paniagua_fbref.com/en/players/2e67fc18/Moises-Paniagua',
 'goal7': 'A_44_Martín Cauteruccio_fbref.com/en/players/6e7d7a7a/Martin-Cauteruccio '}

# 2 loop

In [70]:
# loop fn_match_data
# URLs to loop through
import requests
from bs4 import BeautifulSoup
import pandas as pd
import polars as pl
import time
import random
from tqdm.auto import tqdm
import xlwings as xw
import os

# REMOVE CSV at the START

# desktop or notebook
if os.getenv("COMPUTERNAME")=="T16":
    all_seasons_csv =r"C:\Dropbox\04_SOCCER DATA\00 SOCCER\01 fbref\01 matches\01 fbref_bs\csv\2_all_seasons_urls.csv"
else:
    all_seasons_csv =r"D:\Dropbox\04_SOCCER DATA\00 SOCCER\01 fbref\01 matches\01 fbref_bs\csv\2_all_seasons_urls.csv" 
df_seasons = pd.read_csv(all_seasons_csv)
final_df = pl.DataFrame()
csv_target = r".\csv\3_all_matches.csv"
dfs = []
global counter
counter=-1
frow = 4
# lrow =df.shape[0]
lrow = len(df_seasons)
#lrow = 1
#print(lrow)
for index in tqdm(range(frow, lrow),desc="matches"):
    print("index season",index +1)
    url = df_seasons.iloc[index]['Season URL'] 
    df = fn_get_matchtable(url,index)

    # 2 concat the dataframes
    final_df = pl.concat([final_df, df])
    # save the first time with header
    if index ==0:
        final_df.write_csv(csv_target)
        final_df = pl.DataFrame()
        print("final_df saved for the first time")    
    else:      
        with open(csv_target, mode="ab") as f:
                final_df.write_csv(f)
                final_df.write_csv(f, include_header=False)
                final_df = pl.DataFrame()
                print(f"final_df saved after season {index + 2 }")
# csv
#final_df.write_csv(csv_target)
with open(csv_target, mode="ab") as f:
    final_df.write_csv(f)
    final_df.write_csv(f, include_header=False)

df_opt = pl.read_csv(csv_target)
#duplicates
df_opt = df_opt.unique()
# columns with no values
df_opt = df_opt.filter(df_opt["Score"].is_not_null() & (df_opt["Score"] != ""))
df_opt = df_opt.filter(df_opt["Score"].is_not_null() & (df_opt["Score"] != "Score"))
df_opt.write_csv(csv_target)
df_opt

# # Excel
# xw.view(final_df)




matches:   0%|          | 0/2038 [00:00<?, ?it/s]

index season 6
status: 200, id 6.1_1  season: https://fbref.com/en/comps/14/2019/schedule/
status: 200, id 6.2_2  season: https://fbref.com/en/comps/14/2019/schedule/
status: 200, id 6.3_3  season: https://fbref.com/en/comps/14/2019/schedule/


KeyboardInterrupt: 

In [55]:
final_df


Country,Comp,Season,Round,Wk,Day,Date,Time,Home,xG_h,Score,xG_a,Away,Attendance,Venue,Referee,href,CoachH,CoachA,CaptainH,CaptainA,url_CaptainH,url_CaptainA,goal1,goal2,goal3,goal4,goal5,goal6,goal7,goal8,goal9,goal10,goal11,goal12,goal13,goal14,goal15,goal16,goal17,goal18,goal19,goal20,goal21,goal22,goal23,goal24,goal25
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""""","""Copa Libertado…","""2023""","""Group stage""","""1""","""Tue""","""2023-04-04""","""17:00""","""Alianza Lima""","""0.9""","""0–0""","""1.5""","""Ath Paranaense…","""""","""Estadio Alejan…","""Carlos Ortega""","""https://fbref.…","""Guillermo Sala…","""Paulo Turra""","""Josepmir Balló…","""Thiago Heleno""","""fbref.com/en/p…","""fbref.com/en/p…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""","""Group stage""","""1""","""Tue""","""2023-04-04""","""18:00""","""The Strongest""","""2.2""","""3–1""","""2.8""","""River Plate""","""""","""Estadio Hernan…","""Jesús Valenzue…","""https://fbref.…","""Ismael Recalvo…","""Martín Demiche…","""Adrian Jusino""","""Enzo Pérez""","""fbref.com/en/p…","""fbref.com/en/p…","""H_25_Enrique T…","""H_35_Enrique T…","""H_48_Gonzalo C…","""A_66_Lucas Bel…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""","""Group stage""","""1""","""Tue""","""2023-04-04""","""19:00""","""Argentinos Jun…","""2.1""","""1–0""","""0.7""","""Independiente""","""""","""Estadio Diego …","""Diego Haro""","""https://fbref.…","""Gabriel Milito…","""Martín Anselmi…","""Gabriel Ávalos…","""Cristian Pelle…","""fbref.com/en/p…","""fbref.com/en/p…","""H_46_Javier Ca…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""","""Group stage""","""1""","""Wed""","""2023-04-05""","""17:00""","""SD Aucas""","""1.4""","""2–1""","""0.9""","""Flamengo""","""""","""Estadio Gonzal…","""José Argote""","""https://fbref.…","""César Farías""","""Vítor Pereira""","""Jhonny Quiñóne…","""Everton Ribeir…","""fbref.com/en/p…","""fbref.com/en/p…","""H_58_Erick Cas…","""H_85_Roberto O…","""A_39_Matheus F…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""","""Group stage""","""1""","""Wed""","""2023-04-05""","""19:00""","""Patronato""","""0.7""","""1–2""","""1.4""","""Nacional""","""""","""Estadio Brigad…","""Wagner do Nasc…","""https://fbref.…","""Walter Otta""","""Paulo Autuori""","""Sergio Maximil…","""Sebastián Góme…","""fbref.com/en/p…","""fbref.com/en/p…","""H_2_Mateo Leva…","""A_77_Dorlan Pa…","""A_79_Dorlan Pa…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""""","""Copa Libertado…","""2023""","""Semi-finals""","""""","""Wed""","""2023-09-27""","""21:30""","""Fluminense""","""1.3""","""2–2""","""0.9""","""Internacional""","""67,515""","""Estadio Jornal…","""Darío Herrera""","""https://fbref.…","""Fernando Diniz…","""Eduardo Coudet…","""Nino""","""Alan Patrick""","""fbref.com/en/p…","""fbref.com/en/p…","""H_10_Germán Ca…","""H_78_Germán Ca…","""A_45+5_Hugo Ma…","""A_64_Alan Patr…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…",

In [24]:
# nachzügler
csv_target = r".\csv\3_all_matches.csv"

with open(csv_target, mode="ab") as f:
    final_df.write_csv(f)
    final_df.write_csv(f, include_header=False)

df_opt = pl.read_csv(csv_target)
#duplicates
df_opt = df_opt.unique()
# columns with no values
df_opt = df_opt.filter(df_opt["Score"].is_not_null() & (df_opt["Score"] != ""))
df_opt = df_opt.filter(df_opt["Score"].is_not_null() & (df_opt["Score"] != "Score"))
df_opt.write_csv(csv_target)
df_opt

Country,League,Season,Comp,Round,Wk,Day,Date,Time,Home,xG_h,Score,xG_a,Away,Attendance,Venue,Referee,href,CoachH,CoachA,CaptainH,CaptainA,url_CaptainH,url_CaptainA,goal1,goal2,goal3,goal4,goal5,goal6,goal7,goal8,goal9,goal10,goal11,goal12,goal13,goal14,goal15,goal16,goal17,goal18,goal19,goal20,goal21,goal22,goal23,goal24,goal25
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""""","""Copa Libertado…","""2024""",null,"""First qualifyi…",null,"""Wed""","""2024-02-07""","""20:30""","""Aurora bo""","""0.5""","""1–0""","""1.2""","""Mel""","""""","""Estadio Félix …","""Paulo Zanovell…","""https://fbref.…","""Mauricio Soria…","""Pablo de Muner…","""Captain:Luis R…","""Captain:Bernar…","""fbref.com/en/p…","""fbref.com/en/p…","""H_45+1_Martín …","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2024""",null,"""First qualifyi…",null,"""Wed""","""2024-02-14""","""19:30""","""Melgar pe""","""2.6""","""1–1""","""0.6""","""Aur""","""9,018""","""Estadio de la …","""Andrés Matonte…","""https://fbref.…","""Pablo de Muner…","""Mauricio Soria…","""Captain:Bernar…","""Captain:Luis R…","""fbref.com/en/p…","""fbref.com/en/p…","""H_7_Cristian B…","""A_2_Oswaldo Bl…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""",null,"""Group stage""","""1""","""Wed""","""2023-04-05""","""19:30""","""Sporting Crist…","""0.4""","""1–3""","""4.0""","""Flumine""","""""","""Estadio Nacion…","""Wilmar Roldán""","""https://fbref.…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""",null,"""Group stage""","""2""","""Wed""","""2023-04-19""","""21:30""","""Flamengo br""","""1.9""","""2–0""","""0.1""","""Ñuble""","""48,895""","""Estadio Jornal…","""Wilmar Roldán""","""https://fbref.…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""",null,"""Group stage""","""5""","""Wed""","""2023-06-07""","""19:00""","""Argentinos Jun…","""1.5""","""2–1""","""1.3""","""Liverp""","""""","""Estadio Diego …","""Roberto Pérez""","""https://fbref.…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""""","""Copa Libertado…","""2023""",null,"""Round of 16""","""""","""Thu""","""2023-08-10""","""21:00""","""Racing Club ar…","""1.1""","""3–0""","""0.1""","""Nacio""","""38,731""","""Estadio Presid…","""Jesús Valenzue…","""https://fbref.…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""""","""Copa Libertado…","""2023""",null,"""Quarter-finals…","""""","""Wed""","""2023-08-23""","""21:30""","""Boca Juniors a…","""0.8""","""0–0""","""0.5""","""Racing C""","""""","""Estadio Albert…","""Wilton Sampaio…","""https://fbref.…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""

---
# PFLEGE
### 0 change dtypes

In [ ]:
if df_total.dtypes[df_total.columns.index("Date")] != pl.Date:
    df_total = df_total.with_columns( df_total["Date"].str.strptime(pl.Date, "%d/%m/%Y", strict=False).alias("Date") )

if "Time" in df_total.columns and df_total.dtypes[df_total.columns.index("Time")] != pl.Datetime:
    df_total = df_total.with_columns(df_total["Time"].str.strip_chars().str.strptime(pl.Time, "%H:%M").alias("Time") )


# falls ich die punkte gegen kommas tauschen möchte
# for col_name in df_total.columns:
#     if df_total[col_name].dtype == pl.Utf8:
#         df_total = df_total.with_column( df_tot

### 0 remove duplicates from dataframe

In [ ]:
import polars as pl

# Assuming df is your Polars DataFrame
df_unique = df_total.unique()
df_unique


### 1 sort dataframe

### 2 filter dataframe

In [ ]:
# Assuming `df_total` is your Polars DataFrame
df_filtered = df_total.filter(pl.col("Country") == "")
df_filtered


### 3 dataframe from pl to pd

In [ ]:
import polars as pl 
df_total_pl = pl.from_pandas(df_total) 
df_total_pl
